## Task parameters

In [ ]:


uniqueLott_Nreps= 6      # Unique Lottery Repititions  

# Same-type & mixed type Trials Lottery probabilities
st_refPs = [1]                              # Reference option
st_lottPs = [0.13, 0.22, 0.38, .50, .75]    # Lottery option

money_id = 'Money'
cPlus_id = 'CS+'
cMinus_id = 'CS-'

# Same-type task variables
money_refQs = [1]                               # Euros
money_lottQs = [1, 2, 5, 12, 20]                # Euros

cPlus_refQs = [20]                              # mL of CS+ yogurt 
cPlus_lottQs = [20, 40, 80, 120, 200]           # mL of CS+ yogurt

cMinus_refQs = cPlus_refQs                      # mL of CS- yogurt 
cMinus_lottQs = cPlus_lottQs                    # mL of CS- yogurt 

# Mixed-type task variables
mix_refQs = [.2]                                # Euros
cPlus_Mix_lottQs = [40, 80, 120, 150, 200]      # mL of CS+ yogurt 
cMinus_Mix_lottQs = cPlus_Mix_lottQs            # mL of CS- yogurt 

money_refPars = [money_id, money_refQs, st_refPs]
money_lottPars = [money_id, money_lottQs, st_lottPs]
cPlus_refPars = [cPlus_id, cPlus_refQs, st_refPs]
cPlus_lottPars = [cPlus_id, cPlus_lottQs, st_lottPs]
cMinus_refPars = [cMinus_id, cMinus_refQs, st_refPs]
cMinus_lottPars = [cMinus_id, cMinus_lottQs, st_lottPs]


In [ ]:
# %% Subject specific parameters
money_alpha = 0.7
cPlus_alpha = 0.8
cMinus_alpha = 0.8

money_beta = 1.8
cPlus_beta = money_beta
cMinus_beta = money_beta

cPlus_sFactor = .2
cMinus_sFactor = .2
mix_cPlus_beta = money_beta
mix_cMinus_beta = money_beta

alphas = {money_id : money_alpha,
        cPlus_id : cPlus_alpha,
        cMinus_id : cMinus_alpha}
betas = {money_id : money_beta,
        cPlus_id : cPlus_beta,
        cMinus_id : cMinus_beta,
    'mix '+cPlus_id  : mix_cPlus_beta,
    'mix '+cMinus_id : mix_cMinus_beta
             }

scalingFactors = {
        cPlus_id : cPlus_sFactor,
        cMinus_id : cMinus_sFactor}